In [10]:
import pandas as pd
from pandas import Timestamp
import numpy as np
%matplotlib inline
from datetime import datetime as dt
from matplotlib.pylab import date2num
import time
from pandas.core.frame import DataFrame
from dateutil.parser import parse
import seaborn as sns
from sqlalchemy import create_engine
import time
from datetime import datetime
import json

__Because we failed to upload the json file to s3 services, please keep this notebook and the json files in the same folder.__

## Part 1 - Selection (30 points)
_Identify and describe your dataset, its source, and what appeals
to you about it.  Acquire the data and perform an initial exploration
to determine which themes you wish to explore.  Describe the questions
you want to be able to answer with the data, any concerns you have
about the data, and any challenges you expect to have to overcome._

### 1.Upload the original dataset

In [1]:
!wget https://s3.amazonaws.com/chicagotax/chicago_taxi_trips_2016_01.csv

--2018-12-10 16:58:20--  https://s3.amazonaws.com/chicagotax/chicago_taxi_trips_2016_01.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.136.101
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.136.101|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102582862 (98M) [text/csv]
Saving to: ‘chicago_taxi_trips_2016_01.csv.1’

chicago_taxi_trips_ 100%[===================>]  97.83M  48.4MB/s    in 2.0s    

2018-12-10 16:58:22 (48.4 MB/s) - ‘chicago_taxi_trips_2016_01.csv.1’ saved [102582862/102582862]



In [2]:
!wc -l chicago_taxi_trips_2016_01.csv

 1048575 chicago_taxi_trips_2016_01.csv


In [3]:
!csvcut -n chicago_taxi_trips_2016_01.csv

/bin/sh: csvcut: command not found


### 2.Clean the data using Trifacta
#### Trifacta Wangling Recipe

drop col: pickup_census_tract action: Drop         
drop col: dropoff_census_tract action: Drop               
filter type: lessThan col: Day lessThan: 21 action: Keep           
drop col: trip_end_timestamp action: Drop         
filter type: missing missing: pickup_community_area action: Delete        
filter type: missing missing: dropoff_community_area action: Delete        
filter type: missing missing: fare action: Delete          
filter type: missing missing: company action: Delete            
filter type: missing missing: taxi_id action: Delete            
filter type: missing missing: trip_seconds action: Delete            
filter type: missing missing: trip_miles action: Delete           
merge col: Time,column7 as: 'taxi_time'           
drop col: column7 action: Drop             
drop col: Time action: Drop            
settype col: taxi_id type: Integer                
settype col: company, trip_start_timestamp, Month, Day, pickup_community_area, dropoff_community_area, pickup_latitude, pickup_longitude, dropoff_latitude, dropoff_longitude, taxi_time type: String     

__The results after rifacta Wangling is showing as follows:__

In [2]:
!wget https://s3.amazonaws.com/chicagotax/chicago_taxi_trips.csv

--2018-12-10 16:58:49--  https://s3.amazonaws.com/chicagotax/chicago_taxi_trips.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.131.45
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.131.45|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41770995 (40M) [text/csv]
Saving to: ‘chicago_taxi_trips.csv.1’

chicago_taxi_trips. 100%[===================>]  39.83M  43.0MB/s    in 0.9s    

2018-12-10 16:58:50 (43.0 MB/s) - ‘chicago_taxi_trips.csv.1’ saved [41770995/41770995]



In [5]:
!wc -l chicago_taxi_trips.csv

  348841 chicago_taxi_trips.csv


In [11]:
chi = pd.read_csv('chicago_taxi_trips.csv')

In [12]:
chi.head(5)

,taxi_id,trip_start_timestamp,Month,Day,taxi_time,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude
0,85,2016,1,13,6:15:00AM,180,0.4,24,24,4.50,0.0,0.0,0.0,4.50,Cash,107,199,510,199,510
1,6641,2016,1,6,11:15:00PM,420,0.0,8,28,7.25,0.0,0.0,0.0,7.25,Cash,8,419,615,411,545
2,6078,2016,1,3,7:45:00AM,480,0.1,41,69,9.00,0.0,0.0,0.0,9.00,Cash,107,583,195,278,173
3,221,2016,1,3,4:30:00PM,720,0.0,7,28,10.75,0.0,0.0,0.0,10.75,Cash,101,642,32,411,545
4,5367,2016,1,11,2:00:00PM,420,0.0,28,28,6.75,0.0,0.0,0.0,6.75,Cash,107,158,270,158,270


In [13]:
len(chi)

348840

In [14]:
chi.dtypes

taxi_id                     int64
trip_start_timestamp        int64
Month                       int64
Day                         int64
taxi_time                  object
trip_seconds                int64
trip_miles                float64
pickup_community_area       int64
dropoff_community_area      int64
fare                      float64
tips                      float64
tolls                     float64
extras                    float64
trip_total                float64
payment_type               object
company                     int64
pickup_latitude             int64
pickup_longitude            int64
dropoff_latitude            int64
dropoff_longitude           int64
dtype: object

### 3. Combine the time columns

In [15]:
Mon = chi['Month'].astype(int).astype(str).tolist()
Mon_ = []
for i in Mon:
    i = i.zfill(2)
    Mon_.append(i)
chi['Month'] = DataFrame(Mon_)

In [16]:
Day = chi['Day'].astype(str).tolist()
Day_ = []
for i in Day:
    i = i.zfill(2)
    Day_.append(i)
chi['Day'] = DataFrame(Day_)

In [17]:
chi["trip_start_timestamp"] = chi['trip_start_timestamp'].map(str) + chi['Month'].map(str) + chi['Day'].map(str)

In [18]:
chi = chi.drop(['Month', 'Day'], axis=1)

In [19]:
chi.head(10)

,taxi_id,trip_start_timestamp,taxi_time,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude
0,85,20160113,6:15:00AM,180,0.4,24,24,4.50,0.00,0.0,0.0,4.50,Cash,107,199,510,199,510
1,6641,20160106,11:15:00PM,420,0.0,8,28,7.25,0.00,0.0,0.0,7.25,Cash,8,419,615,411,545
2,6078,20160103,7:45:00AM,480,0.1,41,69,9.00,0.00,0.0,0.0,9.00,Cash,107,583,195,278,173
3,221,20160103,4:30:00PM,720,0.0,7,28,10.75,0.00,0.0,0.0,10.75,Cash,101,642,32,411,545
4,5367,20160111,2:00:00PM,420,0.0,28,28,6.75,0.00,0.0,0.0,6.75,Cash,107,158,270,158,270
5,4251,20160101,7:30:00PM,660,4.1,8,6,11.85,3.00,0.0,1.0,15.85,Credit Card,101,210,470,606,617
6,5132,20160108,7:30:00PM,720,0.2,32,24,12.25,2.00,0.0,1.0,15.25,Credit Card,8,18,610,472,207
7,2647,20160115,7:30:00PM,360,0.0,8,8,6.50,0.00,0.0,0.0,6.50,Cash,107,754,410,474,204
8,373,20160102,5:45:00PM,240,0.6,8,32,4.85,0.00,0.0,1.5,6.35,Cash,109,688,206,744,605
9,5540,20160113,7:00:00PM,600,0.9,8,8,7.00,1.11,0.0,0.0,8.11,Credit Card,92,754,410,167,754


### 4. Replace the masked columns
As we can see in the database, some columns such as latitude and longitude are masked insteated of showing the real value. We have a json file contains the code for these masked columns. To make the following dataset more clear, we encode the masked columns by left join the contents of the json file.

In [20]:
with open('column_remapping.json', 'r') as jsonfile:
    json_string = json.load(jsonfile)

#### Pick up latitude

In [21]:
pickup_lat = json_string['pickup_latitude']
chi['pickup_latitude'] = chi['pickup_latitude'].astype(int)
pickup_latitude = pd.DataFrame.from_dict(pickup_lat, orient='index')
pickup_latitude['pickup_latitude'] =pickup_latitude.index
pickup_latitude.rename(columns={ pickup_latitude.columns[0]: "pickup_latitude1" }, inplace=True)
pickup_latitude.head()

,pickup_latitude1,pickup_latitude
0,41.941422478,0
1,41.920265121,1
2,41.898425258,2
3,42.005608023,3
4,41.884272021,4


In [22]:
pickup_latitude['pickup_latitude'] = pickup_latitude['pickup_latitude'].astype(int)
result = pd.merge(chi, pickup_latitude, how='left', on=['pickup_latitude', 'pickup_latitude'])

In [23]:
result = result.drop(['pickup_latitude'], axis=1)
result.head(5)

,taxi_id,trip_start_timestamp,taxi_time,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_longitude,dropoff_latitude,dropoff_longitude,pickup_latitude1
0,85,20160113,6:15:00AM,180,0.4,24,24,4.50,0.0,0.0,0.0,4.50,Cash,107,510,199,510,41.901206994
1,6641,20160106,11:15:00PM,420,0.0,8,28,7.25,0.0,0.0,0.0,7.25,Cash,8,615,411,545,41.89321636
2,6078,20160103,7:45:00AM,480,0.1,41,69,9.00,0.0,0.0,0.0,9.00,Cash,107,195,278,173,41.794090253
3,221,20160103,4:30:00PM,720,0.0,7,28,10.75,0.0,0.0,0.0,10.75,Cash,101,32,411,545,41.914747305
4,5367,20160111,2:00:00PM,420,0.0,28,28,6.75,0.0,0.0,0.0,6.75,Cash,107,270,158,270,41.874005383


#### Pick up longitude

In [24]:
pickup_long = json_string['pickup_longitude']
chi['pickup_longitude'] = chi['pickup_longitude'].astype(int)
pickup_longitude = pd.DataFrame.from_dict(pickup_long, orient='index')
pickup_longitude['pickup_longitude'] =pickup_longitude.index
pickup_longitude.rename(columns={pickup_longitude.columns[0]: "pickup_longitude1" }, inplace=True)
pickup_longitude.head()

,pickup_longitude1,pickup_longitude
0,-87.811605775,0
1,-87.670621043,1
2,-87.656411531,2
3,-87.672538401,3
4,-87.61626757,4


In [25]:
pickup_longitude['pickup_longitude'] = pickup_longitude['pickup_longitude'].astype(int)
result = pd.merge(result, pickup_longitude, how='left', on=['pickup_longitude', 'pickup_longitude'])

In [26]:
result = result.drop(['pickup_longitude'], axis=1)
result.head(5)

,taxi_id,trip_start_timestamp,taxi_time,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,payment_type,company,dropoff_latitude,dropoff_longitude,pickup_latitude1,pickup_longitude1
0,85,20160113,6:15:00AM,180,0.4,24,24,4.50,0.0,0.0,0.0,4.50,Cash,107,199,510,41.901206994,-87.676355989
1,6641,20160106,11:15:00PM,420,0.0,8,28,7.25,0.0,0.0,0.0,7.25,Cash,8,411,545,41.89321636,-87.63784421
2,6078,20160103,7:45:00AM,480,0.1,41,69,9.00,0.0,0.0,0.0,9.00,Cash,107,278,173,41.794090253,-87.592310855
3,221,20160103,4:30:00PM,720,0.0,7,28,10.75,0.0,0.0,0.0,10.75,Cash,101,411,545,41.914747305,-87.654007029
4,5367,20160111,2:00:00PM,420,0.0,28,28,6.75,0.0,0.0,0.0,6.75,Cash,107,158,270,41.874005383,-87.66351755


#### dropoff_latitude

In [27]:
dropoff_latitude = json_string['dropoff_latitude']
chi['dropoff_latitude'] = chi['dropoff_latitude'].astype(int)
dropoff_latitude = pd.DataFrame.from_dict(dropoff_latitude, orient='index')
dropoff_latitude['dropoff_latitude'] =dropoff_latitude.index
dropoff_latitude.rename(columns={dropoff_latitude.columns[0]: "dropoff_latitude1" }, inplace=True)
dropoff_latitude.head()

,dropoff_latitude1,dropoff_latitude
0,41.941422478,0
1,41.920265121,1
2,41.898425258,2
3,42.005608023,3
4,41.884272021,4


In [28]:
dropoff_latitude['dropoff_latitude'] =dropoff_latitude['dropoff_latitude'].astype(int)
result = pd.merge(result, dropoff_latitude, how='left', on=['dropoff_latitude', 'dropoff_latitude'])

In [29]:
result = result.drop(['dropoff_latitude'], axis=1)
result.head(5)

,taxi_id,trip_start_timestamp,taxi_time,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,payment_type,company,dropoff_longitude,pickup_latitude1,pickup_longitude1,dropoff_latitude1
0,85,20160113,6:15:00AM,180,0.4,24,24,4.50,0.0,0.0,0.0,4.50,Cash,107,510,41.901206994,-87.676355989,41.901206994
1,6641,20160106,11:15:00PM,420,0.0,8,28,7.25,0.0,0.0,0.0,7.25,Cash,8,545,41.89321636,-87.63784421,41.879255084
2,6078,20160103,7:45:00AM,480,0.1,41,69,9.00,0.0,0.0,0.0,9.00,Cash,107,173,41.794090253,-87.592310855,41.763246799
3,221,20160103,4:30:00PM,720,0.0,7,28,10.75,0.0,0.0,0.0,10.75,Cash,101,545,41.914747305,-87.654007029,41.879255084
4,5367,20160111,2:00:00PM,420,0.0,28,28,6.75,0.0,0.0,0.0,6.75,Cash,107,270,41.874005383,-87.66351755,41.874005383


#### dropoff_longitude

In [30]:
dropoff_longitude = json_string['dropoff_longitude']
chi['dropoff_longitude'] = chi['dropoff_longitude'].astype(int)
dropoff_longitude = pd.DataFrame.from_dict(dropoff_longitude, orient='index')
dropoff_longitude['dropoff_longitude'] =dropoff_longitude.index
dropoff_longitude.rename(columns={dropoff_longitude.columns[0]: "dropoff_longitude1" }, inplace=True)
dropoff_longitude.head()

,dropoff_longitude1,dropoff_longitude
0,-87.811605775,0
1,-87.670621043,1
2,-87.656411531,2
3,-87.672538401,3
4,-87.61626757,4


In [31]:
dropoff_longitude['dropoff_longitude'] =dropoff_longitude['dropoff_longitude'].astype(int)
result = pd.merge(result, dropoff_longitude, how='left', on=['dropoff_longitude', 'dropoff_longitude'])
result = result.drop(['dropoff_longitude'], axis=1)
result.head(5)

,taxi_id,trip_start_timestamp,taxi_time,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_latitude1,pickup_longitude1,dropoff_latitude1,dropoff_longitude1
0,85,20160113,6:15:00AM,180,0.4,24,24,4.50,0.0,0.0,0.0,4.50,Cash,107,41.901206994,-87.676355989,41.901206994,-87.676355989
1,6641,20160106,11:15:00PM,420,0.0,8,28,7.25,0.0,0.0,0.0,7.25,Cash,8,41.89321636,-87.63784421,41.879255084,-87.642648998
2,6078,20160103,7:45:00AM,480,0.1,41,69,9.00,0.0,0.0,0.0,9.00,Cash,107,41.794090253,-87.592310855,41.763246799,-87.616134111
3,221,20160103,4:30:00PM,720,0.0,7,28,10.75,0.0,0.0,0.0,10.75,Cash,101,41.914747305,-87.654007029,41.879255084,-87.642648998
4,5367,20160111,2:00:00PM,420,0.0,28,28,6.75,0.0,0.0,0.0,6.75,Cash,107,41.874005383,-87.66351755,41.874005383,-87.66351755


#### dropoff_longitude

In [32]:
company = json_string['company']
chi['company'] = chi['company'].astype(int)
companies = pd.DataFrame.from_dict(company, orient='index')
companies['company'] =companies.index
companies.rename(columns={companies.columns[0]: "company1" }, inplace=True)
companies.head()

,company1,company
0,3623-Arrington Enterprises,0
1,5874 - Sergey Cab Corp.,1
2,5874 - 73628 Sergey Cab Corp.,2
3,Chicago Medallion Management,3
4,3011 - JBL Cab Inc.,4


In [33]:
companies['company'] = companies['company'].astype(int)
result = pd.merge(result, companies, how='left', on=['company', 'company'])
result = result.drop(['company'], axis=1)
result.head(5)

,taxi_id,trip_start_timestamp,taxi_time,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,payment_type,pickup_latitude1,pickup_longitude1,dropoff_latitude1,dropoff_longitude1,company1
0,85,20160113,6:15:00AM,180,0.4,24,24,4.50,0.0,0.0,0.0,4.50,Cash,41.901206994,-87.676355989,41.901206994,-87.676355989,Taxi Affiliation Services
1,6641,20160106,11:15:00PM,420,0.0,8,28,7.25,0.0,0.0,0.0,7.25,Cash,41.89321636,-87.63784421,41.879255084,-87.642648998,Blue Ribbon Taxi Association Inc.
2,6078,20160103,7:45:00AM,480,0.1,41,69,9.00,0.0,0.0,0.0,9.00,Cash,41.794090253,-87.592310855,41.763246799,-87.616134111,Taxi Affiliation Services
3,221,20160103,4:30:00PM,720,0.0,7,28,10.75,0.0,0.0,0.0,10.75,Cash,41.914747305,-87.654007029,41.879255084,-87.642648998,Dispatch Taxi Affiliation
4,5367,20160111,2:00:00PM,420,0.0,28,28,6.75,0.0,0.0,0.0,6.75,Cash,41.874005383,-87.66351755,41.874005383,-87.66351755,Taxi Affiliation Services


In [34]:
result.columns= ['taxi_id',
 'trip_start_timestamp',
 'Time',
 'trip_seconds',
 'trip_miles',
 'pickup_community_area',
 'dropoff_community_area',
 'fare',
 'tips',
 'tolls',
 'extras',
 'trip_total',
 'payment_type',
 'pickup_latitude',
 'pickup_longitude',
 'dropoff_latitude',
 'dropoff_longitude',
 'company']

In [35]:
chi = result

In [36]:
len(chi)

348840

In [37]:
chi.head(10)

,taxi_id,trip_start_timestamp,Time,trip_seconds,trip_miles,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,payment_type,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,company
0,85,20160113,6:15:00AM,180,0.4,24,24,4.50,0.00,0.0,0.0,4.50,Cash,41.901206994,-87.676355989,41.901206994,-87.676355989,Taxi Affiliation Services
1,6641,20160106,11:15:00PM,420,0.0,8,28,7.25,0.00,0.0,0.0,7.25,Cash,41.89321636,-87.63784421,41.879255084,-87.642648998,Blue Ribbon Taxi Association Inc.
2,6078,20160103,7:45:00AM,480,0.1,41,69,9.00,0.00,0.0,0.0,9.00,Cash,41.794090253,-87.592310855,41.763246799,-87.616134111,Taxi Affiliation Services
3,221,20160103,4:30:00PM,720,0.0,7,28,10.75,0.00,0.0,0.0,10.75,Cash,41.914747305,-87.654007029,41.879255084,-87.642648998,Dispatch Taxi Affiliation
4,5367,20160111,2:00:00PM,420,0.0,28,28,6.75,0.00,0.0,0.0,6.75,Cash,41.874005383,-87.66351755,41.874005383,-87.66351755,Taxi Affiliation Services
5,4251,20160101,7:30:00PM,660,4.1,8,6,11.85,3.00,0.0,1.0,15.85,Credit Card,41.892507781,-87.626214906,41.942691844,-87.651770507,Dispatch Taxi Affiliation
6,5132,20160108,7:30:00PM,720,0.2,32,24,12.25,2.00,0.0,1.0,15.25,Credit Card,41.880994471,-87.632746489,41.912364354,-87.675062757,Blue Ribbon Taxi Association Inc.
7,2647,20160115,7:30:00PM,360,0.0,8,8,6.50,0.00,0.0,0.0,6.50,Cash,41.892042136,-87.63186395,41.907520075,-87.6266589,Taxi Affiliation Services
8,373,20160102,5:45:00PM,240,0.6,8,32,4.85,0.00,0.0,1.5,6.35,Cash,41.890922026,-87.618868355,41.884987192,-87.620992913,Choice Taxi Association
9,5540,20160113,7:00:00PM,600,0.9,8,8,7.00,1.11,0.0,0.0,8.11,Credit Card,41.892042136,-87.63186395,41.89503345,-87.619710672,Top Cab Affiliation


__For here, we have finished most data cleaning jobs needed. As some code cells in this notebook is extreme slow, so we opened a new notebook to the the rest jobs.__